## Setup

### Import libraries

In [1]:
from benchmarks.experiments.wrapper_overhead.common.deployment_utility import WrapperOverheadDeploymentUtility

### Setup Common Resources

In [2]:
region = 'us-east-2'
deployment_utility: WrapperOverheadDeploymentUtility = WrapperOverheadDeploymentUtility(region)

## Deploy Associated AWS functions

### DNA Visualization

#### Boto3 Direct Calls

In [12]:
dna_vis_direct_calls_dir = "./dna_visualization/external_database/boto3_only_direct_calls" # Boto3 only direct calls
deployment_utility.deploy_experiment(dna_vis_direct_calls_dir)

Deploying boto3_direct workload
workflow-wo-dna_vis-ed-direct

Deploying wo-dna_vis-ed-direct-visualize
Removing old resources
Old resources for Removed
Creating iam role
Resulting Policy ARN: arn:aws:iam::485595969306:role/wo-dna_vis-ed-direct-visualize-policy
Creating lambda function


#0 building with "default" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 325B done
#1 DONE 0.0s

#2 [internal] load metadata for public.ecr.aws/lambda/python:3.8
#2 DONE 0.2s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [1/6] FROM public.ecr.aws/lambda/python:3.8@sha256:9ae4af1397c3c7ac09aca53cca2621c9fdd5f0a2674f2bf05920888d24883f91
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 108.76kB done
#5 DONE 0.0s

#6 [2/6] COPY requirements.txt ./
#6 CACHED

#7 [3/6] RUN pip3 install --no-cache-dir -r requirements.txt
#7 CACHED

#8 [4/6] COPY app.py ./
#8 DONE 0.0s

#9 [5/6] COPY src ./src
#9 DONE 0.0s

#10 [6/6] COPY multi_x_serverless ./multi_x_serverless
#10 DONE 0.0s

#11 exporting to image
#11 exporting layers 0.0s done
#11 writing image sha256:fc27a18479f29a61e936116a5238425cedad20cc475fa97239bd4f61fa8e4e5b done
#11 naming to docker.io/library/wo-dna_vis-ed-dir

Login Succeeded
The push refers to repository [485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-dna_vis-ed-direct-visualize]
ff099a48b25c: Preparing
1b64c10ef09e: Preparing
a85c2f3154ef: Preparing
8e1b406f394f: Preparing
3a89be36bfcd: Preparing
0a20c8da2344: Preparing
05788b7af5ed: Preparing
706e59713fa4: Preparing
052242efe275: Preparing
274a79fec6a9: Preparing
6849347e354b: Preparing
05788b7af5ed: Waiting
706e59713fa4: Waiting
052242efe275: Waiting
274a79fec6a9: Waiting
0a20c8da2344: Waiting
6849347e354b: Waiting
8e1b406f394f: Layer already exists
3a89be36bfcd: Layer already exists
0a20c8da2344: Layer already exists
05788b7af5ed: Layer already exists
052242efe275: Layer already exists
706e59713fa4: Layer already exists
274a79fec6a9: Layer already exists
6849347e354b: Layer already exists
a85c2f3154ef: Pushed
1b64c10ef09e: Pushed
ff099a48b25c: Pushed


Successfully pushed Docker image 485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-dna_vis-ed-direct-visualize:latest to ECR.


latest: digest: sha256:4c58e2814fd8d2211dc1dadf052ca2201821ceb7aa665a888ab65e2b8f8136b2 size: 2624
Resulting Function ARN: arn:aws:lambda:us-east-2:485595969306:function:wo-dna_vis-ed-direct-visualize

Deploying wo-dna_vis-ed-direct-get_input
Removing old resources
Old resources for Removed
Creating iam role
Resulting Policy ARN: arn:aws:iam::485595969306:role/wo-dna_vis-ed-direct-get_input-policy
Creating lambda function


#0 building with "default" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 325B done
#1 DONE 0.0s

#2 [internal] load metadata for public.ecr.aws/lambda/python:3.8
#2 DONE 0.1s

#3 [internal] load .dockerignore
#3 transferring context:
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [1/6] FROM public.ecr.aws/lambda/python:3.8@sha256:9ae4af1397c3c7ac09aca53cca2621c9fdd5f0a2674f2bf05920888d24883f91
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 106.67kB done
#5 DONE 0.0s

#6 [2/6] COPY requirements.txt ./
#6 CACHED

#7 [3/6] RUN pip3 install --no-cache-dir -r requirements.txt
#7 CACHED

#8 [4/6] COPY app.py ./
#8 DONE 0.0s

#9 [5/6] COPY src ./src
#9 DONE 0.0s

#10 [6/6] COPY multi_x_serverless ./multi_x_serverless
#10 DONE 0.0s

#11 exporting to image
#11 exporting layers 0.0s done
#11 writing image sha256:1118d7a328602f35a6ba1a00416a629acbdcc116b625e6cfe99e12c9689c4209 done
#11 naming to docker.io/

Login Succeeded
The push refers to repository [485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-dna_vis-ed-direct-get_input]
a332f03f55c4: Preparing
ff793b0c6b78: Preparing
11c1e1de5b7d: Preparing
6fe8485efa46: Preparing
34352bc5dfd0: Preparing
0a20c8da2344: Preparing
05788b7af5ed: Preparing
706e59713fa4: Preparing
052242efe275: Preparing
274a79fec6a9: Preparing
6849347e354b: Preparing
706e59713fa4: Waiting
6849347e354b: Waiting
052242efe275: Waiting
274a79fec6a9: Waiting
0a20c8da2344: Waiting
05788b7af5ed: Waiting
34352bc5dfd0: Layer already exists
6fe8485efa46: Layer already exists
0a20c8da2344: Layer already exists
05788b7af5ed: Layer already exists
706e59713fa4: Layer already exists
052242efe275: Layer already exists
274a79fec6a9: Layer already exists
6849347e354b: Layer already exists
11c1e1de5b7d: Pushed
ff793b0c6b78: Pushed
a332f03f55c4: Pushed


Successfully pushed Docker image 485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-dna_vis-ed-direct-get_input:latest to ECR.


latest: digest: sha256:f99057dffa33a529ca8bea8a2700588d06e1bfe6b9330f03395d2e5a25f8131d size: 2623
Resulting Function ARN: arn:aws:lambda:us-east-2:485595969306:function:wo-dna_vis-ed-direct-get_input
Completed deployment of workflow-wo-dna_vis-ed-direct




True

#### Boto3 SNS calls

In [13]:
dna_vis_sns_calls_dir = "./dna_visualization/external_database/boto3_only_sns" # Boto3 only sns calls
deployment_utility.deploy_experiment(dna_vis_sns_calls_dir)

Deploying boto3_sns workload
workflow-wo-dna_vis-ed-direct_sns

Deploying wo-dna_vis-ed-direct_sns-visualize
Removing old resources
Old resources for Removed
Creating iam role
Resulting Policy ARN: arn:aws:iam::485595969306:role/wo-dna_vis-ed-direct_sns-visualize-policy
Creating lambda function


#0 building with "default" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 325B done
#1 DONE 0.0s

#2 [internal] load metadata for public.ecr.aws/lambda/python:3.8
#2 DONE 0.2s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [1/6] FROM public.ecr.aws/lambda/python:3.8@sha256:9ae4af1397c3c7ac09aca53cca2621c9fdd5f0a2674f2bf05920888d24883f91
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 108.84kB done
#5 DONE 0.0s

#6 [2/6] COPY requirements.txt ./
#6 CACHED

#7 [3/6] RUN pip3 install --no-cache-dir -r requirements.txt
#7 CACHED

#8 [4/6] COPY app.py ./
#8 DONE 0.0s

#9 [5/6] COPY src ./src
#9 DONE 0.0s

#10 [6/6] COPY multi_x_serverless ./multi_x_serverless
#10 DONE 0.0s

#11 exporting to image
#11 exporting layers 0.0s done
#11 writing image sha256:9efb8bef44800ec686158f8c041b0313792f0fde8e230f630aa739071bd3e107 done
#11 naming to docker.io/library/wo-dna_vis-ed-dir

Login Succeeded
The push refers to repository [485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-dna_vis-ed-direct_sns-visualize]
55a8a8f9310d: Preparing
b44f79ea1cdc: Preparing
e3c6852c69dd: Preparing
8e1b406f394f: Preparing
3a89be36bfcd: Preparing
0a20c8da2344: Preparing
05788b7af5ed: Preparing
706e59713fa4: Preparing
052242efe275: Preparing
274a79fec6a9: Preparing
6849347e354b: Preparing
05788b7af5ed: Waiting
706e59713fa4: Waiting
052242efe275: Waiting
274a79fec6a9: Waiting
6849347e354b: Waiting
0a20c8da2344: Waiting
8e1b406f394f: Layer already exists
3a89be36bfcd: Layer already exists
05788b7af5ed: Layer already exists
0a20c8da2344: Layer already exists
052242efe275: Layer already exists
706e59713fa4: Layer already exists
6849347e354b: Layer already exists
274a79fec6a9: Layer already exists
b44f79ea1cdc: Pushed
e3c6852c69dd: Pushed
55a8a8f9310d: Pushed


Successfully pushed Docker image 485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-dna_vis-ed-direct_sns-visualize:latest to ECR.


latest: digest: sha256:8d71d31ad66c28d903da230b6c84bc2ec063b76d1895e96e86712a24817dc694 size: 2624
Resulting Function ARN: arn:aws:lambda:us-east-2:485595969306:function:wo-dna_vis-ed-direct_sns-visualize
Creating sns topic and subscribing lambda function
Resulting Topic ARN: arn:aws:sns:us-east-2:485595969306:wo-dna_vis-ed-direct_sns-visualize-sns_topic
Resulting Subscription ARN: arn:aws:sns:us-east-2:485595969306:wo-dna_vis-ed-direct_sns-visualize-sns_topic:b1d6bd5e-1149-4c3d-a89d-6c7e5598c9d0

Deploying wo-dna_vis-ed-direct_sns-get_input
Removing old resources
Old resources for Removed
Creating iam role
Resulting Policy ARN: arn:aws:iam::485595969306:role/wo-dna_vis-ed-direct_sns-get_input-policy
Creating lambda function


#0 building with "default" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 325B done
#1 DONE 0.0s

#2 [internal] load metadata for public.ecr.aws/lambda/python:3.8
#2 DONE 0.1s

#3 [internal] load .dockerignore
#3 transferring context:
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [1/6] FROM public.ecr.aws/lambda/python:3.8@sha256:9ae4af1397c3c7ac09aca53cca2621c9fdd5f0a2674f2bf05920888d24883f91
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 107.14kB done
#5 DONE 0.0s

#6 [2/6] COPY requirements.txt ./
#6 CACHED

#7 [3/6] RUN pip3 install --no-cache-dir -r requirements.txt
#7 CACHED

#8 [4/6] COPY app.py ./
#8 DONE 0.0s

#9 [5/6] COPY src ./src
#9 DONE 0.0s

#10 [6/6] COPY multi_x_serverless ./multi_x_serverless
#10 DONE 0.0s

#11 exporting to image
#11 exporting layers 0.0s done
#11 writing image sha256:39d309232ec739e1f1f397f537ea52c80998db9938d898e0d06980fde1d7dd8e done
#11 naming to docker.io/

Login Succeeded
The push refers to repository [485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-dna_vis-ed-direct_sns-get_input]
f51a8ed27576: Preparing
e29f925bb07a: Preparing
f590117785db: Preparing
6fe8485efa46: Preparing
34352bc5dfd0: Preparing
0a20c8da2344: Preparing
05788b7af5ed: Preparing
706e59713fa4: Preparing
05788b7af5ed: Waiting
0a20c8da2344: Waiting
052242efe275: Preparing
274a79fec6a9: Preparing
6849347e354b: Preparing
706e59713fa4: Waiting
6849347e354b: Waiting
052242efe275: Waiting
274a79fec6a9: Waiting
34352bc5dfd0: Layer already exists
6fe8485efa46: Layer already exists
05788b7af5ed: Layer already exists
0a20c8da2344: Layer already exists
706e59713fa4: Layer already exists
052242efe275: Layer already exists
274a79fec6a9: Layer already exists
6849347e354b: Layer already exists
e29f925bb07a: Pushed
f590117785db: Pushed
f51a8ed27576: Pushed


Successfully pushed Docker image 485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-dna_vis-ed-direct_sns-get_input:latest to ECR.


latest: digest: sha256:2a175bff215b5d1a91c8f55e4d15c602321b373b7834a6a68cad24a81a36aa4c size: 2623
Resulting Function ARN: arn:aws:lambda:us-east-2:485595969306:function:wo-dna_vis-ed-direct_sns-get_input
Creating sns topic and subscribing lambda function
Resulting Topic ARN: arn:aws:sns:us-east-2:485595969306:wo-dna_vis-ed-direct_sns-get_input-sns_topic
Resulting Subscription ARN: arn:aws:sns:us-east-2:485595969306:wo-dna_vis-ed-direct_sns-get_input-sns_topic:b132e2f6-25ce-4eb4-99e7-f8f9662b9561
Completed deployment of workflow-wo-dna_vis-ed-direct_sns




True

#### AWS Step Function

In [ ]:
dna_vis_aws_step_functions_dir = "./dna_visualization/external_database/aws_step_function" # AWS Step Functions
deployment_utility.deploy_experiment(dna_vis_aws_step_functions_dir)

#### Multi-X

- For MultiX, please set it up according to readme contained in the file. "./dna_visualization/external_database/multi_x"